## Reference Implementation

### E2E Architecture

![use_case_flow](assets/conversationai-e2e-flow.PNG)

### Solution setup
Use the following cell to change to the correct kernel. Then check that you are in the `stock` kernel. If not, navigate to `Kernel > Change kernel > Python [conda env:stock]`. Note that the cell will remain with * but you can continue running the following cells.

In [ ]:
%%javascript
Jupyter.notebook.session.restart({kernel_name: 'conda-env-customer_chatbot_intel-py'})

Instructions to download and prepare this dataset for benchmarking using these scripts can be found by in the `data` directory [here](data/README.md).

### Running the Benchmarks for Training

Benchmarking for training can be done using the python script `run_training.py`.

The script *reads and preprocesses the data*, *trains an joint clasification and entity recognition model*, and *predicts on unseen test data* using the trained model, while also reporting on the execution time for these 3 steps.  ***Optionally, the script can also save the trained model weights, which is necessary to run the inference benchmarks***.

The run benchmark script takes the following arguments:

```shell
usage: run_training.py [-h] [-l LOGFILE] [-i] [-s SAVE_MODEL_DIR] [--save_onnx]

optional arguments:
  -h, --help            show this help message and exit
  -l LOGFILE, --logfile LOGFILE
                        log file to output benchmarking results to
  -i, --intel           use intel accelerated technologies where available
  -s SAVE_MODEL_DIR, --save_model_dir SAVE_MODEL_DIR
                        directory to save model under
  --save_onnx           also export an ONNX model
```

#### Training the Initial Model

In [ ]:
!python -m intel_extension_for_pytorch.cpu.launch $WORKSPACE/src/run_training.py --logfile $OUTPUT_DIR/logs/intel.log -s $DATA_DIR/saved_models/intel -d $DATA_DIR/atis-2/

### Running the Benchmarks for Inference

Benchmarking for inference for Pytorch (.pt) models can be done using the python script `run_inference.py`.

`run_inference.py` : runs inference benchmarks using models optimized using the Intel® Extension for PyTorch*.

The `run_inference.py` script takes the following arguments:

```shell
usage: run_inference.py [-h] -s SAVED_MODEL_DIR [--is_jit] [--is_inc_int8] [-i] [-b BATCH_SIZE] [-l LENGTH]
                        [--logfile LOGFILE] [-n N_RUNS]

optional arguments:
  -h, --help            show this help message and exit
  -s SAVED_MODEL_DIR, --saved_model_dir SAVED_MODEL_DIR
                        directory of saved model to benchmark.
  --is_jit              if the model is torchscript. defaults to False.
  --is_inc_int8         saved model dir is a quantized int8 model. defaults to False.
  -i, --intel           use intel accelerated technologies. defaults to False.
  -b BATCH_SIZE, --batch_size BATCH_SIZE
                        batch size to use. defaults to 200.
  -l LENGTH, --length LENGTH
                        sequence length to use. defaults to 512.
  --logfile LOGFILE     logfile to use.
  -n N_RUNS, --n_runs N_RUNS
                        number of trials to test. defaults to 100.
```

In [ ]:
python -m intel_extension_for_pytorch.cpu.launch $WORKSPACE/src/run_inference.py -s $DATA_DIR/saved_models/intel --batch_size 200 --length 512 --n_runs 5 --logfile $OUTPUT_DIR/logs/intel.log -d $DATA_DIR/atis-2/

#### Intel® Neural Compressor Quantization

A trained model from the `run_training.py` script above can be quantized 
using [Intel® Neural Compressor](https://www.intel.com/content/www/us/en/developer/tools/oneapi/neural-compressor.html) 
through the `run_quantize_inc.py` script.  This converts the model from FP32 to INT8 while trying to 
maintain a specified level of accuracy specified via a `config.yaml` file. A simple `config.yaml` has been 
provided for basic accuracy aware quantization though several further options exist and can be explored in the link above.


```shell
usage: run_quantize_inc.py [-h] -s SAVED_MODEL -o OUTPUT_DIR [-l LENGTH] [-q QUANT_SAMPLES] -c INC_CONFIG

optional arguments:
  -h, --help            show this help message and exit
  -s SAVED_MODEL, --saved_model SAVED_MODEL
                        saved pytorch (.pt) model to quantize.
  -o OUTPUT_DIR, --output_dir OUTPUT_DIR
                        directory to save quantized model to.
  -l LENGTH, --length LENGTH
                        sequence length to use. defaults to 512.
  -q QUANT_SAMPLES, --quant_samples QUANT_SAMPLES
                        number of samples to use for quantization. defaults to 100.
  -c INC_CONFIG, --inc_config INC_CONFIG
                        INC conf yaml.
```

A workflow of training -> INC quantization -> inference benchmarking may look like

In [ ]:
python $WORKSPACE/src/run_quantize_inc.py -s $DATA_DIR/saved_models/intel/convai.pt -o $DATA_DIR/saved_models/intel_int8/ -c $CONFIG_DIR/config.yml -d $DATA_DIR/atis-2/

In [ ]:
python -m intel_extension_for_pytorch.cpu.launch $WORKSPACE/src/run_inference.py -s $DATA_DIR/saved_models/intel_int8/ -b 1 -n 1000 --is_inc_int8 --logfile $OUTPUT_DIR/logs/intel.log -d $DATA_DIR/atis-2/